In [1]:
import time
import random
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
#数据预处理函数
def deal(data):         #对性别列做值替换
    data.loc[data['Sex'] == 'M',['Sex']] = 1 #对性别列做值替换
    data.loc[data['Sex'] == 'F',['Sex']] = -1
    data.loc[data['Sex'] == 'I',['Sex']] = 0
    return data
def Normaliza(df):  #归一化函数
    std_scaler = lambda x : (x-x.min())/x.std()
    df = df.apply(std_scaler)#赋回原数组
    return df
#数据分析
def check(data):
    for i in origin_feature:
        print(i,data[i].index,data[i].dtype)
    return

In [4]:
#模型建立

#折线图绘制多个曲线
def plot_Gradientdescent(label,iters_num,temp,num):

    plt.figure(figsize=(10,7))
    t=temp.tolist()
    for i in range(num):
        plt.plot(range(1,iters_num+1), t[i+1], label=label[i])
    plt.legend() # 显示图例
    plt.title('Eigenvalue line chart')
    plt.xlabel('x')
    plt.ylabel('y')
 
    plt.show()

#正规方程法求特征值w
def Normal_Equation(x,y):
    x=np.array(x)
    y=np.array(y.values).T
    x = np.hstack((np.ones((x.shape[0],1)),x))
    
    xMat = np.mat(x)
    yMat = np.mat(y).T
    xTx = xMat.T * xMat                            #w=(XT*X)^(-1)*XT*Y
    if np.linalg.det(xTx) == 0.0:
        print("矩阵为奇异矩阵,不能求逆")
        return
    w = xTx.I * xMat.T * yMat
    return w

#梯度下降法求特征值w
def GradientDescent(x,y,alpha,iters_num):
    label=x.columns.values
    x=np.array(x)
    y=np.array(y.values)
    x = np.hstack((np.ones((x.shape[0],1)),x)) #添加常数列

    m = len(y)
    w=np.zeros((x.shape[1],1))
    n = len(w)
    
    temp = np.matrix(np.zeros((n,iters_num)))   # 暂存每次迭代计算的特征值w，转化为矩阵形式
    
    J_history = np.zeros((iters_num,1)) #记录每次迭代计算的代价值
    
    for i in range(iters_num):  # 遍历迭代次数    
        h = np.dot(x,w)     # 计算内积，matrix可以直接乘
        temp[:,i] = w - ((alpha/m)*(np.dot(np.transpose(x),h-y)))   #梯度的计算
        w = temp[:,i]
        J_history[i] = ComputerCost(x,y,w)      #调用计算代价函数

    #plot_Gradientdescent(label,iters_num,temp,x.shape[1]-1)
    return w,J_history  

#代价函数
def ComputerCost(x,y,w):
    m = len(y)
    J = 0
    J = np.dot(np.transpose(np.dot(x,w)-y),(np.dot(x,w)-y))/(2*m)
    return J

#误差大小评价函数
def ComputerError(x,y,w):
    x=np.array(x)
    y=np.array(y)
    x = np.hstack((np.ones((x.shape[0],1)),x))
    m=x.shape[0] 
    result=np.array(np.dot(x,w)-y)
    err=(np.multiply(result,result).sum())/(2*m)
    return err

def plot_Cost(cost):
    plt.figure(figsize=(10,7))
    t=cost.T.tolist()
    #fig.title('Result Analysis')
    plt.plot(range(1,iters_num+1), t[0], label='Cost Fuction')
    plt.legend() # 显示图例
    plt.title('Cost Fuction Line chart')
    plt.xlabel('x')
    plt.ylabel('y') 
    plt.show()
    
def KFold(train, numVal = 10):
    xrownum=train.shape[0]
    xcolnum=train.shape[1]
    xcolname=list(train)
    trainArr=np.array(train)                                                                                             
    indexList = list(range(xrownum))  
    #打乱次序
    random.shuffle(indexList)  
    result=[]
    for i in range(numVal):     #交叉验证numVal次
        #训练集
        trainX = pd.DataFrame(columns=xcolname)
        #测试集
        testX = pd.DataFrame(columns=xcolname)
        step=int(xrownum/numVal)
        start=i*step
        end=(i+1)*step-1
        testX=pd.DataFrame(dict(zip(xcolname,trainArr[indexList[start:end]].T)))
        if start:
            trainX=pd.DataFrame(dict(zip(xcolname,trainArr[indexList[0:start-1]].T)))
        trainX=trainX.append(pd.DataFrame(dict(zip(xcolname,trainArr[indexList[end+1:xrownum]].T))),ignore_index=True)
        result.append([trainX,testX])   
    return result

def KFoldGradientDescent(train,numVal):
    kf=KFold(train,numVal)
    i=0
    w=np.zeros((numVal,train.shape[1]))
    print(w.shape)
    errorMat = np.zeros((numVal,1))   
    for curtrain,curtest in kf:

        trainX=curtrain.iloc[:,0:8]
        trainY=curtrain[['Rings']]
        testX=curtest.iloc[:,0:8]
        testY=curtest[['Rings']]    
        learning_rate=0.01
        iters_num=1600
        print('正在进行第'+str(i+1)+'次训练....')
        wMat,J = GradientDescent(trainX,trainY,learning_rate,iters_num)  
        w[i]=wMat.T
        print(wMat.T)
        errorMat[i] = ComputerError(testX,testY,wMat)   #统计误差值                      
        print(errorMat[i])
        i += 1

    print(numVal,'折交叉验证平均特征值为:')
    print(w.mean(axis=0))
    print(numVal,'折交叉验证平均误差为:')
    print(errorMat.mean())
    
def main():
    #读取数据
    print("开始读取训练数据...")
    t0 = time.time()
    train = pd.read_csv('abalone.csv',low_memory=False)
    print('读取数据一共用时{}秒'.format(time.time() - t0))
    # 数据预处理
    print("开始进行数据预处理...")
    t0 = time.time()
    #check(train)
    train=deal(train)
    #df=train.iloc[:,0:8]
    #train.iloc[:,0:8]=Normaliza(df)
    print('数据预处理一共用时{}秒'.format(time.time() - t0))
    numVal=10
    print(numVal,'折交叉验证......')
    print('梯度下降法')
    KFoldGradientDescent(train,numVal)
if __name__ =='__main__':
    main()

开始读取训练数据...
读取数据一共用时0.007977962493896484秒
开始进行数据预处理...
数据预处理一共用时0.007016897201538086秒
10 折交叉验证......
梯度下降法
(10, 9)
正在进行第1次训练....
[[ 5.4269638  -0.05403012  2.5478289   2.06189393  0.80642123  2.24296029
  -0.20848707  0.38355118  1.43788235]]
[3.50367999]
正在进行第2次训练....
[[ 5.43348886 -0.10523305  2.54580309  2.06073069  0.82069754  2.23730761
  -0.20618594  0.3922269   1.44241515]]
[3.72262223]
正在进行第3次训练....
[[ 5.42534516 -0.07103753  2.54904036  2.06062367  0.82195986  2.25285728
  -0.20420127  0.38226386  1.44307254]]
[3.4364371]
正在进行第4次训练....
[[ 5.43369678 -0.10516376  2.55684755  2.0659922   0.81777959  2.2443886
  -0.20836611  0.37098988  1.44324174]]
[3.3219425]
正在进行第5次训练....
[[ 5.39779185 -0.06533799  2.54273125  2.05539975  0.8136796   2.29031762
  -0.18349311  0.37552252  1.46501055]]
[3.62284519]
正在进行第6次训练....
[[ 5.41672743 -0.08455542  2.55045008  2.06548717  0.81829118  2.26654964
  -0.20897408  0.39449405  1.45108711]]
[3.21432189]
正在进行第7次训练....
[[ 5.44069195 -0.09731     2